In [11]:
!pip install mecab-python3
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!pip install neologdn
!pip install jaconv

import collections
from collections import namedtuple
import jaconv
import MeCab
import neologdn
import numpy as np
import pandas as pd
import re

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
ln: failed to create symbolic link '/usr/local/etc/mecabrc': File exists


In [ ]:
%cd drive/My Drive

/content/drive/My Drive


In [ ]:
df = pd.read_csv('./@cosme_1120.csv').reset_index()

In [ ]:
change_reviews = []
for review in df['review']:
  review = neologdn.normalize(review)
  review = review.replace('@', 'アット')
  review = review.replace('＠', 'アット')
  review = review.replace('cosme', 'コスメ')
  review = review.replace('■', '')
  #review = review.replace('リピ', 'リピート')
  review = review.replace('good', '良い')
  review = review.replace('コスパ', 'コストパフォーマンス')
  review = review.replace('いいです', '良い')
  review = review.replace('よかった', '良い')
  review = review.replace('します', 'する')
  review = review.replace('しません', 'しない')
  review = re.split('[。.！!?？!?♪★☆(´･`)*^]', review)
  change_reviews.append(' '.join(review))
reviews_df = pd.DataFrame(change_reviews, columns=['review'])

In [ ]:
#http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt
#上のURL開いてダウンロードする
#テキストファイル読み込み
with open('Japanese.txt', 'r', encoding='utf-8') as f:
  stopwords = [w.strip() for w in f]
  stopwords = list(stopwords)
stopwords.append('*')

with open ('negapoji_jp.txt', 'r', encoding = 'cp932') as f:
  #単語・読み仮名・品詞・スコアに分割してリストとして格納
  negapoji_list = [negapoji_word.replace('\n', '').split(':') for negapoji_word in  f.readlines()]
negapoji_df = pd.DataFrame(negapoji_list, columns = ['base_form', 'reading', 'pos', 'score'])
#jaconvを使って読み仮名を全てカタカナに変換
negapoji_df['reading'] = negapoji_df['reading'].apply(lambda x : jaconv.hira2kata(x))
#なぜか読みや品詞まで同じなのに、異なるスコアが割り当てられていたものがあったので重複を削除
negapoji_df = negapoji_df[~negapoji_df[['base_form', 'reading', 'pos']].duplicated()]

In [ ]:
class NegaPojiAnalysis():
  def __init__(self, reviews_df, negapoji_list, stopwords):
    self.reviews_df = reviews_df
    self.negapoji_list = negapoji_list
    self.stopwords = stopwords
    self.path = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    self.tagger = MeCab.Tagger(self.path)

  def tokenize(self):
    word_list = []
    for i, review in enumerate(self.reviews_df['review']):
        node = self.tagger.parseToNode(review)
        while node:
          surface = node.surface
          pos = node.feature.split(",")[0]
          pos_detail1 = node.feature.split(",")[1]
          pos_detail2 = node.feature.split(",")[2]
          pos_detail3 = node.feature.split(",")[3]
          infl_type = node.feature.split(",")[4]  
          infl_form = node.feature.split(",")[5]
          base_form = node.feature.split(",")[6]
          if len(node.feature.split(",")) >= 8:
            reading = node.feature.split(",")[7]
          if pos == '名詞' or pos == '動詞' or pos == '形容詞' or pos == '副詞':
            word_list.append([i, surface, pos, pos_detail1, pos_detail2, pos_detail3, infl_type, infl_form, base_form, reading])
          node = node.next
    word_df = pd.DataFrame(word_list, columns = ['index', 'surface', 'pos', 'pos_detail1', 'pos_detail2', 'pos_detail3', 'infl_type', 'infl_form', 'base_form', 'reading'])
    word_df['pos'] = word_df['pos'].apply(lambda negapoji_list : negapoji_list.split(',')[0])
    return word_df

  def generate_negapoji_score(self, word_df, negapoji_df):
    score_result_df = pd.merge(word_df, negapoji_df, on = ['base_form', 'pos', 'reading'], how = 'left')
    return score_result_df

  def generate_review_score(self, score_result_df):
    result = []
    for i in range(len(score_result_df['index'].unique())):
        temp_df = score_result_df[score_result_df['index'] == i]
        text = self.reviews_df['review'][i]
        #text = ''.join(list(temp_df['word']))
        score = temp_df['score'].astype(float).sum()
        #スコアをスコアが付与されている単語数で割った値を算出
        score_r = score/temp_df['score'].astype(float).count()
        result.append([i, text, score,score_r])
    result_df = pd.DataFrame(result, columns = ['index', 'review', '累計スコア', '標準化スコア']).sort_values(by = '標準化スコア', ascending=False).reset_index(drop = True)
    return result_df

In [ ]:
negapoji_analysis = NegaPojiAnalysis(reviews_df, negapoji_list, stopwords)
parse_text_df = negapoji_analysis.tokenize()
score_result_df = negapoji_analysis.generate_negapoji_score(parse_text_df, negapoji_df)
review_result_df = negapoji_analysis.generate_review_score(score_result_df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
review_result_df

,index,review,累計スコア,標準化スコア
0,692,first time user and so excited to use it beca...,0.999995,0.999995
1,520,良い,0.999995,0.999995
2,517,香りが良いです,0.999995,0.999995
3,491,リピしてます コストパフォーマンス良い,0.999995,0.999995
4,690,肌荒れへの効果があり良かったです,0.938205,0.938205
...,...,...,...,...
895,196,使い続ける理由が見当たらないです ・・ 笑,-1.568902,-0.784451
896,288,使い始めて1週間 使わない時と違いがあるので、しばらく続けてみます,-1.579045,-0.789523
897,333,口コミよくて買ってみましたが思っていたほどではなかったです もう少し潤いあるかと思っていたの...,-4.803684,-0.800614
898,715,みずみずしい水のような茶色のテクスチャーでしたとろみがないので少し使いづらかったです,-3.315282,-0.828820
